![Two data scientists working on a dashboard.](hr-image-small.png)
A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

## **Project Instructions**

The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called **`ds_jobs_transformed`** that stores the data in **`customer_train.csv`** much more efficiently. Specifically, they have set the following requirements:

- Columns containing categories with only two factors must be stored as Booleans (**`bool`**).
- Columns containing integers only must be stored as 32-bit integers (**`int32`**).
- Columns containing floats must be stored as 16-bit floats (**`float16`**).
- Columns containing nominal categorical data must be stored as the **`category`** data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
- The DataFrame should be filtered to only contain students with _10 or more years_ of experience at companies with at _least 1000 employees_, as their recruiter base is suited to more experienced professionals at enterprise companies.

If you call **`.info()`** or **`.memory_usage()`** methods on **`ds_jobs`** and **`ds_jobs_transformed`** after you've preprocessed it, you should notice a substantial decrease in memory usage.

In [ ]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [ ]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# View ds_jobs_transformed information
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [ ]:
ds_jobs_transformed.relevant_experience.value_counts()

# int type
# training_hours,student_id

# float type
# city_development_index

# bool type
# relevant_experience,job_change

# ordered category
# company_size,last_new_job,experience

# category type
# enrolled_university,education_level,major_discipline,company_type,gender,city

Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64

*   ### Columns containing integers only must be stored as 32-bit integers (**`int32`**).




In [ ]:
# Convert data type to int32
columns = ['training_hours', 'student_id']
for col in columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')

ds_jobs_transformed[columns].dtypes

training_hours    int32
student_id        int32
dtype: object

- ### Columns containing nominal categorical data must be stored as the category data type.

In [ ]:
# Convert data type to category
columns = ['education_level','major_discipline','company_type','gender','city']
for col in columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

ds_jobs_transformed[columns].dtypes

education_level     category
major_discipline    category
company_type        category
gender              category
city                category
dtype: object

- ### Columns containing floats must be stored as 16-bit floats (**`float16`**).

In [ ]:
# Convert data type to float16
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')
ds_jobs_transformed['city_development_index'].dtype

dtype('float16')

- ### Columns containing categories with only two factors must be stored as Booleans (**`bool`**).

In [ ]:
# codes = ds_jobs_transformed['relevant_experience'].astype('category').cat.codes
# codes

# Create a mapping dictionary with 0 and 1 and using the map() method to transform the relevant_experience column
map_dict = {'No relevant experience' : 0, 'Has relevant experience' : 1}
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map(map_dict)

# Convert columns to bool data type
columns = ['relevant_experience', 'job_change']
for col in columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('bool')

ds_jobs_transformed[columns].dtypes

relevant_experience    bool
job_change             bool
dtype: object

- ### Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.

In [ ]:
# Create a dictionary of columns with ordered categories
ordered_categories = {
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Converting column data type to ordered category type
for column in ordered_categories.keys():
    ds_jobs_transformed[column] = ds_jobs_transformed[column].astype(pd.CategoricalDtype(categories = ordered_categories[column], ordered=True))

- ### The DataFrame should be filtered to only contain students with _10 or more years_ of experience at companies with _at least 1000 employees_, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [ ]:
 # Filtering on ordered categorical columns
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]

- ### Call **`.info()`** or **`.memory_usage()`** methods on **`ds_jobs`** and **`ds_jobs_transformed`** to check if there is a substantial decrease in memory usage.

In [ ]:
# Compare memory usage for ds_jobs and ds_jobs_transformed
print(ds_jobs.info(verbose = False))
print()
print(ds_jobs_transformed.info(verbose = False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Columns: 14 entries, student_id to job_change
dtypes: float64(2), int64(2), object(10)
memory usage: 2.0+ MB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Columns: 14 entries, student_id to job_change
dtypes: bool(2), category(9), float16(1), int32(2)
memory usage: 69.5 KB
None


## Summary:
- ### Exploratory data analysis
- ### Converting integers, floats, and unordered categories
- ### Converting ordered categories
- ### Filtering on ordered categorical columns

## Conclusion:
### After preprocessing and filtering the **`ds_jobs_transformed`** data based on the         above requirements and comparing it with the **`ds_jobs`** data using the                   **`.info()`** method, we noticed a substantial decrease in memory usage from **`2.0+ MB`** to **`69.5 KB`**. Hence, memory usage or allocation can be efficiently manage by modifying its storage data type.